## GBP/TRY data set

In [16]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import datetime


In [17]:
data_GBP_TR = pd.read_csv("csv files/GBP_TRY.csv")
data_GBP_TR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      2900 non-null   object 
 1   Price     2900 non-null   float64
 2   Open      2900 non-null   float64
 3   High      2900 non-null   float64
 4   Low       2900 non-null   float64
 5   Vol.      1044 non-null   object 
 6   Change %  2900 non-null   object 
dtypes: float64(4), object(3)
memory usage: 158.7+ KB


In [18]:
df = data_GBP_TR.drop(['Open','High','Low','Vol.','Change %'], axis=1)

In [19]:
df = df.rename(columns={'Date': 'Tarih', 'Price': 'Şimdi'})
df

,Tarih,Şimdi
0,12/30/2022,22.6072
1,12/29/2022,22.5301
2,12/28/2022,22.4888
3,12/27/2022,22.4395
4,12/26/2022,22.5061
...,...,...
2895,01/06/2012,2.8974
2896,01/05/2012,2.9166
2897,01/04/2012,2.9342
2898,01/03/2012,2.9305


## LSTM

In [20]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[22.6072 22.5301 22.4888 22.4395 22.5061] 22.5047
[22.5301 22.4888 22.4395 22.5061 22.5047] 22.451
[22.4888 22.4395 22.5061 22.5047 22.451 ] 22.5649
[22.4395 22.5061 22.5047 22.451  22.5649] 22.715
[22.5061 22.5047 22.451  22.5649 22.715 ] 22.6433
[22.5047 22.451  22.5649 22.715  22.6433] 22.622
[22.451  22.5649 22.715  22.6433 22.622 ] 22.6922
[22.5649 22.715  22.6433 22.622  22.6922] 23.1319
[22.715  22.6433 22.622  22.6922 23.1319] 22.9513
[22.6433 22.622  22.6922 23.1319 22.9513] 22.8621
[22.622  22.6922 23.1319 22.9513 22.8621] 22.8559
[22.6922 23.1319 22.9513 22.8621 22.8559] 22.793
[23.1319 22.9513 22.8621 22.8559 22.793 ] 22.7334
[22.9513 22.8621 22.8559 22.793  22.7334] 22.6039
[22.8621 22.8559 22.793  22.7334 22.6039] 22.6973
[22.8559 22.793  22.7334 22.6039 22.6973] 22.891
[22.793  22.7334 22.6039 22.6973 22.891 ] 22.8066
[22.7334 22.6039 22.6973 22.891  22.8066] 22.4115
[22.6039 22.6973 22.891  22.8066 22.4115] 22.2773
[22.6973 22.891  22.8066 22.4115 22.2773] 22.2543
[22.8

In [21]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df['Şimdi'][0],df['Şimdi'][1],df['Şimdi'][2],df['Şimdi'][3],df['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Epoch 1/200
91/91 [==============================] - 1s 5ms/step - loss: 7.5838
Epoch 2/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0775
Epoch 3/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0767
Epoch 4/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0762
Epoch 5/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0752
Epoch 6/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0762
Epoch 7/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0665
Epoch 8/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0766
Epoch 9/200
91/91 [==============================] - 1s 6ms/step - loss: 0.0764
Epoch 10/200
91/91 [==============================] - 1s 7ms/step - loss: 0.0873
Epoch 11/200
91/91 [==============================] - 1s 6ms/step - loss: 0.0657
Epoch 12/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0671
Epoch 13/200
91/91 [=================

## Daily

In [22]:
x = datetime.datetime.now()
print("Prediction:",round(yhat[0][0],3), "₺ Sterlin/TRY in", x.strftime('%x'))

Prediction: 22.579 ₺ Sterlin/TRY in 01/01/23


## Weekly

In [23]:
x_input = array([df['Şimdi'][0], df['Şimdi'][1],df['Şimdi'][2], df['Şimdi'][3], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],df['Şimdi'][2], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], float(yhat_4),float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)


yhat = float(yhat)
yhat_2 = float(yhat_2)
yhat_3 = float(yhat_3)
yhat_4 = float(yhat_4)
yhat_5 = float(yhat_5)


In [24]:
print(f"Monday: {round(yhat,3)} Tuesday : {round(yhat_2,3)} Wednesday: {round(yhat_3,3)} Thursday: {round(yhat_4,3)} Friday: {round(yhat_5,3)}  $ Sterlin/TRY")


Monday: 22.579 Tuesday : 22.665 Wednesday: 22.623 Thursday: 22.625 Friday: 22.624  $ Sterlin/TRY


## Monthly

In [25]:
import array
exchange = array.array('f', [])

for i in range(len(df)):
    if i % 5 == 0:  # it equals to weekly
        exchange.append(df['Şimdi'][i])


In [26]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[22.6072 22.5047 22.622  22.8559 22.891 ] 22.475000381469727
[22.5047 22.622  22.8559 22.891  22.475 ] 22.090999603271484
[22.622  22.8559 22.891  22.475  22.091 ] 21.97960090637207
[22.8559 22.891  22.475  22.091  21.9796] 21.118499755859375
[22.891  22.475  22.091  21.9796 21.1185] 21.571800231933594
[22.475  22.091  21.9796 21.1185 21.5718] 21.00860023498535
[22.091  21.9796 21.1185 21.5718 21.0086] 20.729799270629883
[21.9796 21.1185 21.5718 21.0086 20.7298] 20.574100494384766
[21.1185 21.5718 21.0086 20.7298 20.5741] 20.645999908447266
[21.5718 21.0086 20.7298 20.5741 20.646 ] 19.989299774169922
[21.0086 20.7298 20.5741 20.646  19.9893] 20.83169937133789
[20.7298 20.5741 20.646  19.9893 20.8317] 21.12700080871582
[20.5741 20.646  19.9893 20.8317 21.127 ] 20.91710090637207
[20.646  19.9893 20.8317 21.127  20.9171] 21.31290054321289
[19.9893 20.8317 21.127  20.9171 21.3129] 21.38909912109375
[20.8317 21.127  20.9171 21.3129 21.3891] 21.75200080871582
[21.127  20.9171 21.3129 21.3891

In [27]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)


Epoch 1/200
18/18 [==============================] - 1s 6ms/step - loss: 22.5412
Epoch 2/200
18/18 [==============================] - 0s 6ms/step - loss: 3.3303
Epoch 3/200
18/18 [==============================] - 0s 6ms/step - loss: 0.4593
Epoch 4/200
18/18 [==============================] - 0s 6ms/step - loss: 0.3427
Epoch 5/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2825
Epoch 6/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2786
Epoch 7/200
18/18 [==============================] - 0s 6ms/step - loss: 0.3350
Epoch 8/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2759
Epoch 9/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2966
Epoch 10/200
18/18 [==============================] - 0s 8ms/step - loss: 0.3104
Epoch 11/200
18/18 [==============================] - 0s 8ms/step - loss: 0.2723
Epoch 12/200
18/18 [==============================] - 0s 7ms/step - loss: 0.2694
Epoch 13/200
18/18 [================

In [28]:
print("Next Month -> Prediction:", round(float(yhat[0][0]),3), "₺ Sterlin/TRY in", x.strftime('%B'))


Next Month -> Prediction: 22.483 ₺ Sterlin/TRY in January
